In [10]:
# I cannot pip install on the Hub
import sys
import os
sys.path.append(os.path.abspath('/home/jovyan/work/hetdex/packs/private-het-data/src'))

import het_cov

SyntaxError: incomplete input (630842710.py, line 1)